In [92]:
import pandas as pd
import datetime
import time
import numpy as np

In [93]:
from coinmarketcap import Market

In [94]:
coinmarketcap = Market()

In [95]:
def format_new_data(df, coin):
    df.columns = [str(col) + '_{}'.format(coin) for col in df.columns]
    df['last_updated_{}'.format(coin)] = df['last_updated_{}'.format(coin)].apply(lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))
    return df

In [96]:
def rsi(x, coin):
    delta = x['price_usd_{}'.format(coin)].iloc[-15:] - x['price_usd_{}'.format(coin)].iloc[-15:].shift(1)
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    RSI = 100.0 - (100.0 / (1 + (float((sum(up[1:])/14))/(sum(down[1:].abs()/14)))))
    return RSI

def MACD(x, coin):
#     x['26 ema'] = 
    tsix = x['price_usd_{}'.format(coin)].ewm(span = 26).mean()
    twelve = x['price_usd_{}'.format(coin)].ewm(span = 12).mean()
    MACD = twelve - tsix
    signal = MACD.ewm(span=9).mean()
    return MACD[-1]

def MACD_signal(x, coin):
    signal = x['MACD_{}'.format(coin)].ewm(span=9).mean()
    return signal[-1]
    
def stoch_rsi(x, coin):
    rsi = x['rsi_{}'.format(coin)].iloc[-1]
    rsi_window = x['rsi_{}'.format(coin)].iloc[-14:]
    rsi_max = float(rsi_window.max())
    rsi_min = rsi_window.min()
    return (rsi - rsi_min) / (rsi_max - rsi_min)

def cci(x, coin):
    x['typical_price'] = ((x['price_usd_{}'.format(coin)] - x['price_usd_{}'.format(coin)].shift()) / 2\
                         + x['price_usd_{}'.format(coin)].shift())
    x['rolling_tp'] = x['typical_price'].rolling(14).mean()
    mean_deviation = (x['typical_price'].iloc[-14:] - x['rolling_tp'].iloc[-14:]).abs().sum() / 14
    return (x['typical_price'].iloc[-1] - x['rolling_tp'].iloc[-1]) / (0.15 - mean_deviation)

def williams_r(x, coin):
    lowest_low = x['price_usd_{}'.format(coin)].iloc[-14:].min()
    highest_high = x['price_usd_{}'.format(coin)].iloc[-14:].max()
    return (highest_high - float(x['price_usd_{}'.format(coin)].iloc[-1]))/(highest_high - lowest_low) * -100

In [97]:
# def rules for trading

def MACD_trade(x, coin):
    if x['MACD_{}'.format(coin)].iloc[-1] > x['signal_{}'.format(coin)].iloc[-1]:
        return 'bullish'
    elif x['MACD_{}'.format(coin)].iloc[-1] < x['signal_{}'.format(coin)].iloc[-1]:
        return 'bearish'
    else:
        return None
    
def CCI_trade(x, coin):    
    if x['cci_{}'.format(coin)].iloc[-1] < -100:
        return 'bearish'
    elif x['cci_{}'.format(coin)].iloc[-1] > 100:
        return 'bullish'
    else:
        return None
    
def Stoch_RSI_trade(x, coin):
    if x['stoch_rsi_{}'.format(coin)].iloc[-1] > .8:
        return 'bullish'
    elif x['stoch_rsi_{}'.format(coin)].iloc[-1] < .2:
        return 'bearish'
    else:
        return None
    
def Williams_R_trade(x, coin):
    if x['r_{}'.format(coin)].iloc[-1] < -80:
        return 'bullish'
    elif x['r_{}'.format(coin)].iloc[-1] > -20:
        return 'bearish'
    else:
        return None

In [101]:
def buy_sell(x, coin):
    count_buy = 0
    count_sell = 0
    for indicator in [x['MACD_trade_{}'.format(coin)].iloc[-1], x['CCI_trade_{}'.format(coin)].iloc[-1], \
                      x['Stoch_RSI_trade_{}'.format(coin)].iloc[-1], x['Williams_R_trade_{}'.format(coin)].iloc[-1]]:
        if indicator == 'bullish':
            count_buy += 1
        if indicator == 'bearish':
            count_buy += 1
    final_indicator = count_buy - count_sell
    if final_indicator > 3:
        return 'strong buy'
    elif final_indicator >= 2:
        return 'buy'
    elif final_indicator < 2 & final_indicator > 0:
        return 'watch buy'
    elif final_indicator <-3:
        return 'strong sell'
    elif final_indicator <= -2:
        return 'sell'
    elif final_indicator >-2 & final_indicator < 0:
        return 'watch sell'
    else:
        return 'no signal'

In [102]:
def execute_data_pull():
    ethereum = pd.DataFrame(coinmarketcap.ticker('Ethereum', limit=3, convert='USD'))
    ethereum = format_new_data(ethereum, 'ethereum')
    bitcoin = pd.DataFrame(coinmarketcap.ticker('Bitcoin', limit=3, convert='USD'))
    bitcoin = format_new_data(bitcoin, 'bitcoin')
    bitcoin_ethereum = pd.merge(bitcoin, ethereum, left_index=True, right_index=True)
    return bitcoin_ethereum

In [103]:
f = open('csvfile.csv','w')
x = 0
while x < 40:
#     time_start = datetime.datetime.now()
    btc_eth = execute_data_pull()
    btc_eth['last_updated_bitcoin'] = datetime.datetime.now() + datetime.timedelta(minutes = x)
    btc_eth['price_usd_bitcoin'] = np.random.randint(20)
    btc_eth['price_usd_ethereum'] = np.random.randint(20)
    btc_eth.set_index('last_updated_bitcoin', inplace=True)
    try:
        df = df.append(btc_eth)
    except:
        df = btc_eth
        df = df.reindex(columns = [df.columns.tolist() + ['rsi_ethereum','MACD_ethereum', \
                                                          'rsi_bitcoin', 'MACD_bitcoin',\
                                                         'stoch_rsi_ethereum', 'stoch_rsi_bitcoin',\
                                                         'cci_ethereum', 'cci_bitcoin',\
                                                         'r_ethereum', 'r_bitcoin',\
                                                         'signal_ethereum', 'signal_bitcoin',\
                                                         'MACD_trade_ethereum', 'MACD_trade_bitcoin',\
                                                         'CCI_trade_ethereum', 'CCI_trade_bitcoin',\
                                                         'Stoch_RSI_trade_ethereum', 'Stoch_RSI_trade_bitcoin',\
                                                         'Williams_R_trade_ethereum', 'Williams_R_trade_bitcoin',\
                                                         'buy_sell_ethereum', 'buy_sell_bitcoin']], fill_value=None)
    if x > 31:
        df.loc[df.index[-1], 'rsi_ethereum'] = rsi(df, 'ethereum')
        df.loc[df.index[-1], 'MACD_ethereum'] = MACD(df, 'ethereum')
        df.loc[df.index[-1], 'signal_ethereum'] = MACD_signal(df, 'ethereum')
        df.loc[df.index[-1], 'rsi_bitcoin'] = rsi(df, 'bitcoin')
        df.loc[df.index[-1], 'MACD_bitcoin'] = MACD(df, 'bitcoin')
        df.loc[df.index[-1], 'signal_bitcoin'] = MACD_signal(df, 'bitcoin')
        df.loc[df.index[-1], 'stoch_rsi_bitcoin'] = stoch_rsi(df, 'bitcoin')
        df.loc[df.index[-1], 'stoch_rsi_ethereum'] = stoch_rsi(df, 'ethereum')
        df.loc[df.index[-1], 'cci_bitcoin'] = cci(df, 'bitcoin')
        df.loc[df.index[-1], 'cci_ethereum'] = cci(df, 'ethereum')
        df.loc[df.index[-1], 'r_bitcoin'] = williams_r(df, 'bitcoin')
        df.loc[df.index[-1], 'r_ethereum'] = williams_r(df, 'ethereum')
        df.loc[df.index[-1], 'MACD_trade_ethereum'] = MACD_trade(df, 'ethereum')
        df.loc[df.index[-1], 'MACD_trade_bitcoin'] = MACD_trade(df, 'bitcoin')
        df.loc[df.index[-1], 'CCI_trade_ethereum'] = CCI_trade(df, 'ethereum')
        df.loc[df.index[-1], 'CCI_trade_bitcoin'] = CCI_trade(df, 'bitcoin')
        df.loc[df.index[-1], 'Stoch_RSI_trade_ethereum'] = Stoch_RSI_trade(df, 'ethereum')
        df.loc[df.index[-1], 'Stoch_RSI_trade_bitcoin'] = Stoch_RSI_trade(df, 'bitcoin')
        df.loc[df.index[-1], 'Williams_R_trade_ethereum'] = Williams_R_trade(df, 'ethereum')
        df.loc[df.index[-1], 'Williams_R_trade_bitcoin'] = Williams_R_trade(df, 'bitcoin')
        df.loc[df.index[-1], 'buy_sell_ethereum'] = buy_sell(df, 'ethereum')
        df.loc[df.index[-1], 'buy_sell_bitcoin'] = buy_sell(df, 'bitcoin')
#     f.write(df.iloc[-1])
    df = df.iloc[1:]
    time.sleep(1)
#     time.sleep(5 - time_start - datetime.datetime.now())
    x += 1
f.close()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:26: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
from bittrex.bittrex import *

my_bittrex = Bittrex("<my_api_key>", "<my_api_secret>", api_version="<API_V1_1> or <API_V2_0>")

my_bittrex.get_balance('ETH')